In [2]:
import bsscraper
import importlib
importlib.reload(bsscraper)

<module 'bsscraper' from '/Users/stevennewton/Documents/GitHub/MLB_Statcast/notebooka/bsscraper.py'>

In [ ]:
# Database for two years and two teams
bsscraper.make_database('NYY_NYM_19_20.db', [2019,2020], ['NYY,NYM'])

In [3]:
# Csv for 2020 Yankees only.
bsscraper.savant_call(2020, "NYY", csv=True)

('NYY_2020.csv', <http.client.HTTPMessage at 0x7fc722c342b0>)

In [93]:
import sqlite3
import pandas as pd

# Connect to, query, and load database into dataframe.
conn = sqlite3.connect('NYY_NYM_19_20_.db')
df = pd.read_sql_query("""
                        SELECT player_name, pitcher, game_year, game_date, game_type, home_team, away_team, inning, pitch_type,
                        pitch_name, pitch_number, release_speed, release_spin_rate, vx0, vy0, vz0, events, type,
                        balls, strikes, zone, release_pos_x, release_pos_z, pfx_x, pfx_z, plate_x, plate_z, 
                        release_extension, ax, ay, az, launch_speed, launch_speed_angle
                        FROM statcast
                        WHERE pitch_type IS NOT NULL;
                       """, conn)
conn.close()
df

,player_name,pitcher,game_year,game_date,game_type,home_team,away_team,inning,pitch_type,pitch_name,...,pfx_x,pfx_z,plate_x,plate_z,release_extension,ax,ay,az,launch_speed,launch_speed_angle
0,Aroldis Chapman,547973,2019,2019-10-19,L,HOU,NYY,9,SL,Slider,...,-0.85,0.49,0.56,2.55,6.4,-8.785636,25.968637,-26.851318,104.9,6.0
1,Aroldis Chapman,547973,2019,2019-10-19,L,HOU,NYY,9,SL,Slider,...,-0.75,0.22,-0.33,2.29,6.6,-7.734415,25.202487,-29.335449,NaN,NaN
2,Aroldis Chapman,547973,2019,2019-10-19,L,HOU,NYY,9,FF,4-Seam Fastball,...,0.32,1.56,1.26,3.48,7.0,3.974430,34.979047,-9.519166,NaN,NaN
3,Aroldis Chapman,547973,2019,2019-10-19,L,HOU,NYY,9,FF,4-Seam Fastball,...,0.16,1.61,1.45,3.38,6.5,1.799115,35.611654,-8.575509,NaN,NaN
4,Aroldis Chapman,547973,2019,2019-10-19,L,HOU,NYY,9,FF,4-Seam Fastball,...,0.12,1.50,0.47,3.82,6.7,1.772995,33.490644,-10.930272,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68463,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-0.31,1.33,1.56,3.31,7.0,-6.125120,31.063964,-12.569235,NaN,NaN
68464,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-0.74,1.39,-0.42,2.72,6.9,-11.223951,35.264626,-11.820192,NaN,NaN
68465,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-0.61,1.28,1.56,3.34,7.0,-10.585942,32.953592,-13.606505,NaN,NaN
68466,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-0.59,1.32,0.82,2.41,7.2,-9.820061,32.272418,-12.570117,NaN,NaN


game_type

    Type of Game. E = Exhibition, S = Spring Training, R = Regular Season, F = Wild Card, D = Divisional Series, L = League Championship Series, W = World Series

In [94]:
#To verify this has every pitch, I'll estimate how many rows I expect for 2 teams over 2 years
est_pitches_per_game=150
games=162+60   # shortened 2020 season
f'approximately {est_pitches_per_game*games*2} pitches'

'approximately 66600 pitches'

In [95]:
#Drop columns with less than 50 percent data
df = df.dropna(axis=1, thresh=.5*df.shape[0])

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68468 entries, 0 to 68467
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player_name        68468 non-null  object 
 1   pitcher            68468 non-null  int64  
 2   game_year          68468 non-null  int64  
 3   game_date          68468 non-null  object 
 4   game_type          68468 non-null  object 
 5   home_team          68468 non-null  object 
 6   away_team          68468 non-null  object 
 7   inning             68468 non-null  int64  
 8   pitch_type         68468 non-null  object 
 9   pitch_name         68468 non-null  object 
 10  pitch_number       68468 non-null  int64  
 11  release_speed      68468 non-null  float64
 12  release_spin_rate  67729 non-null  float64
 13  vx0                68468 non-null  float64
 14  vy0                68468 non-null  float64
 15  vz0                68468 non-null  float64
 16  type               684

In [97]:
#Observations in 2019 and 2020 data
df.shape

(68468, 30)

In [98]:
#Using only 2020 data and more accurte Hawk-Eye system
df2 = df[df.game_year==2020]

In [99]:
df2.shape

(18616, 30)

In [100]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18616 entries, 49852 to 68467
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player_name        18616 non-null  object 
 1   pitcher            18616 non-null  int64  
 2   game_year          18616 non-null  int64  
 3   game_date          18616 non-null  object 
 4   game_type          18616 non-null  object 
 5   home_team          18616 non-null  object 
 6   away_team          18616 non-null  object 
 7   inning             18616 non-null  int64  
 8   pitch_type         18616 non-null  object 
 9   pitch_name         18616 non-null  object 
 10  pitch_number       18616 non-null  int64  
 11  release_speed      18616 non-null  float64
 12  release_spin_rate  18615 non-null  float64
 13  vx0                18616 non-null  float64
 14  vy0                18616 non-null  float64
 15  vz0                18616 non-null  float64
 16  type              